In [1]:
import requests
import gpxpy


trace_route_url = "https://valhalla.mapzen.com/trace_route?api_key=mapzen-jLrDBSP&"
trace_attributes_url = "https://valhalla.mapzen.com/trace_attributes?api_key=mapzen-jLrDBSP&"

gpx_shape = []

with open("./192_B_AP_2017_07_19_03_44_08.gpx", "r") as gpx_file:
    #parse gpx for mapzen
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coord = {}
                coord["lon"] = point.longitude
                coord["lat"] = point.latitude
                gpx_shape.append(coord)
                

In [2]:
#étape 1 : on passe dans trace_attributes             

shape_txt = str(gpx_shape)
shape_txt = shape_txt.replace("'","\"")
data = '{"shape":'+ shape_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 
ta1 = requests.post(trace_attributes_url,  data = data)

output_polyline = ta1.json()['shape']

In [3]:
#six degrees of precision in valhalla
inv = 1.0 / 1e6;

#decode an encoded string
def decode(encoded):
  decoded = []
  previous = [0,0]
  i = 0
  #for each byte
  while i < len(encoded):
    #for each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      #keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      #get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    #scale by the precision and chop off long coords also flip the positions so
    #its the far more standard lon,lat instead of lat,lon
    #decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    decoded.append({'lon': float('%.6f' % (ll[1] * inv)), 'lat': float('%.6f' % (ll[0] * inv))})
  #hand back the list of coordinates
  return decoded




In [4]:
#étape 2: on passe dans trace_route

shape_2_txt = str(decode(output_polyline))
shape_2_txt = shape_2_txt.replace("'","\"")
trace_route_data = '{"shape":'+ shape_2_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 

tr = requests.post(trace_route_url,  data = trace_route_data)
output_polyline2 = tr.json()['trip']['legs'][0]['shape']

#output_polyline2

In [5]:
#étape 3: on repasse dans trace_attribute

shape_3_txt = str(decode(output_polyline2))
shape_3_txt = shape_3_txt.replace("'","\"")
trace_attributes_data = '{"shape":'+ shape_3_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 

ta = requests.post(trace_attributes_url,  data = trace_attributes_data)
#ta.json()


In [6]:
osm_way_list=[]
for a_way in ta.json()['edges']:
    osm_way_list.append(a_way['way_id'])

osm_way_list

#fabrication de la requête overpass
overpass_query_list = ['way('+str(id)+');(._;>;);out meta; ' for id in osm_way_list]

' '.join(overpass_query_list)

#print(len(osm_way_list))

'way(43584844);(._;>;);out meta;  way(24385695);(._;>;);out meta;  way(24385695);(._;>;);out meta;  way(24385710);(._;>;);out meta;  way(24385710);(._;>;);out meta;  way(43626025);(._;>;);out meta;  way(43626025);(._;>;);out meta;  way(43626025);(._;>;);out meta;  way(43626025);(._;>;);out meta;  way(43626025);(._;>;);out meta;  way(210561293);(._;>;);out meta;  way(43626051);(._;>;);out meta;  way(307289972);(._;>;);out meta;  way(307289961);(._;>;);out meta;  way(307289960);(._;>;);out meta;  way(307289962);(._;>;);out meta;  way(307289962);(._;>;);out meta;  way(307289962);(._;>;);out meta;  way(210561290);(._;>;);out meta;  way(29660529);(._;>;);out meta;  way(29660529);(._;>;);out meta;  way(29660529);(._;>;);out meta;  way(308467362);(._;>;);out meta;  way(308467362);(._;>;);out meta;  way(308467362);(._;>;);out meta;  way(308467362);(._;>;);out meta;  way(308467362);(._;>;);out meta;  way(29970723);(._;>;);out meta;  way(29970723);(._;>;);out meta;  way(29970723);(._;>;);out met

Même avec plusieurs passes, le résultat est incomplet u_u